## Chapter 13: Multi-layer Perceptrons 

# 13.0 Introduction

As we first saw in Section 11.1 single layer neural network units - unlike polynomials and other fixed kernels - have internal parameters that allow each element to take on a variety of shapes.  In this Chapter we expand on that introduction extensively, disussing multi-layer neural networks - often also referred to *multi-layer perceptrons*, *feedforward neural networks*, or *fully connected networks*.  